# 1.6 搭建运行环境
- 首先需要在聚宽平台注册试用账号, 获取到账号密码
- 安装 `jqdatasdk` 包,直接 `pip install jqdatasdk` 即可

In [2]:
from jqdatasdk import *
import os
from dotenv import load_dotenv
import pandas as pd
import datetime

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)

# 加载用户名和密码环境变量
load_dotenv()

username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]

# 验证
auth(username, password)

# 获取当前可调用次数
print(f"验证成功, 剩余可调用次数为: {get_query_count()}")

auth success 
验证成功, 剩余可调用次数为: {'total': 1000000, 'spare': 981077}


# 2.4 使用 JQData 查询行情数据

In [ ]:
# 查询所有的股票信息
all_stocks_df = get_all_securities(['stock'])
all_stocks_df

In [ ]:
# 查询讯飞的股价信息
iflytek_df = get_price('002230.XSHE', start_date='2021-7-7 9:00:00', end_date='2021-7-7 11:30:00', frequency='1m', panel=False)
iflytek_df

# 2.6 使用 resample 函数转换时间序列

In [ ]:
iflytek_df_2020 = get_price('002230.XSHE', start_date='2020-1-1', end_date='2021-6-30', frequency='daily', panel=False)


iflytek_df_month = pd.DataFrame()
iflytek_df_month['open'] = iflytek_df_2020['open'].resample('M').first()
iflytek_df_month['close'] = iflytek_df_2020['close'].resample('M').last()
iflytek_df_month['max'] = iflytek_df_2020['high'].resample('M').max()
iflytek_df_month['min'] = iflytek_df_2020['low'].resample('M').min()
iflytek_df_month['volume'] = iflytek_df_2020['volume'].resample('M').sum()
iflytek_df_month['money'] = iflytek_df_2020['money'].resample('M').sum()
iflytek_df_month

# 2.7 使用 JQData 查询财务指标

In [ ]:
finance_df: pd.DataFrame = get_fundamentals(query(indicator), date="2021-01-05")
#df.to_csv('finance.csv')


#基于盈利指标选股, eps, operating_profit, inc_total_revenue_year_on_year
finance_df = finance_df[(finance_df['eps'] > 0) & (finance_df["operating_profit"] > 284197086.3) & (finance_df['roe'] > 11) & (finance_df['inc_total_revenue_year_on_year'] > 10)]
finance_df

# 2.8 使用 JQData 查询估值指标

相对估值法
![](https://raw.githubusercontent.com/logerrors/pic/main/picgo/CleanShot%202021-07-10%20at%2012.01.11%402x.png)

In [ ]:
current_valuation = get_fundamentals(query(valuation), statDate=datetime.date.today())
current_valuation



# 2.9 使用财务数据计算

## 第一题 使用 Python 计算贵州茅台的最新市值数据
![](https://raw.githubusercontent.com/logerrors/pic/main/picgo/CleanShot%202021-07-10%20at%2013.19.49%402x.png)

In [6]:
# 贵州茅台的代码为 600519.XSHE
maotai_valuation = get_fundamentals(query(valuation).filter(
    valuation.code.in_(['600519.XSHG'])
), statDate=datetime.date.today())
maotai_valuation["market_cap"]

0    24773.6016
Name: market_cap, dtype: float64

## 第二题 使用 Python 计算贵州茅台的市盈率(静态)

In [7]:
maotai_valuation_income = get_fundamentals(query(valuation, income).filter(
    valuation.code.in_(['600519.XSHG'])
))
static_pe = maotai_valuation_income['market_cap'][0]/maotai_valuation_income['net_profit'][0]
static_pe

1.6770613667751506e-06